<a href="https://colab.research.google.com/github/FunDuckich/CycleGAN_for_TST_problem/blob/master/DataVectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install navec

In [3]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2024-11-30 17:37:11--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  11.5MB/s    in 5.4s    

2024-11-30 17:37:17 (9.28 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [4]:
import pandas as pd
import numpy as np
import torch
from navec import Navec
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from collections import defaultdict
import seaborn as sb

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv("/content/drive/MyDrive/CycleGAN_for_TST_problem/data.csv")

## Подготовка эмбеддингов слов и знаков пунктуации

In [7]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [8]:
embeddings = list()
for word in navec.vocab.words:
        embeddings.append(navec[word])
embeddings = np.array(embeddings)
embeddings.shape

(500002, 300)

In [9]:
target_dim = 10
pca = PCA(n_components=target_dim)
reduced_embeddings = pca.fit_transform(embeddings)
reduced_embeddings.shape

(500002, 10)

In [10]:
reduced_navec = {word: np.array(reduced_embeddings[i], dtype=np.float16) for i, word in enumerate(navec.vocab.words)}

In [11]:
punkt_vectors = {
    ".": np.array([0.0] * (target_dim - 1) + [1.0], dtype=np.float16),
    "!": np.array([0.0] * (target_dim - 1) + [0.9], dtype=np.float16),
    "?": np.array([0.0] * (target_dim - 1) + [0.8], dtype=np.float16),
    ",": np.array([0.0] * (target_dim - 1) + [0.5], dtype=np.float16),
    ":": np.array([0.0] * (target_dim - 1) + [0.6], dtype=np.float16),
    "–": np.array([0.0] * (target_dim - 1) + [0.4], dtype=np.float16),
    "-": np.array([0.0] * (target_dim - 1) + [0.4], dtype=np.float16),
    "—": np.array([0.0] * (target_dim - 1) + [0.4], dtype=np.float16),
    "«": np.array([0.0] * (target_dim - 1) + [0.3], dtype=np.float16),
    "»": np.array([0.0] * (target_dim - 1) + [0.3], dtype=np.float16),
}

## Изучение размеров токенизированных предложений

In [ ]:
vector_lengths = defaultdict(int)
for line in df.lit_text:
    length = len(word_tokenize(line))
    vector_lengths[length] += 1

In [ ]:
vector_lengths

defaultdict(int,
            {7: 84802,
             8: 79480,
             6: 89589,
             15: 49055,
             23: 26562,
             36: 6069,
             9: 73886,
             12: 60025,
             13: 56422,
             14: 52738,
             28: 17560,
             20: 33937,
             16: 45230,
             5: 84973,
             11: 64251,
             22: 28770,
             33: 10500,
             19: 36029,
             3: 45155,
             34: 8923,
             29: 15840,
             24: 24644,
             27: 19045,
             17: 42153,
             31: 13184,
             21: 31160,
             10: 68863,
             25: 22635,
             30: 14577,
             18: 39337,
             35: 7644,
             4: 72618,
             37: 4986,
             39: 2887,
             2: 6129,
             26: 20726,
             32: 11792,
             41: 1484,
             38: 3920,
             40: 2165,
             48: 59,
             42: 99

In [13]:
max_vector_length = 40

## Векторизация литературы

In [26]:
lit_tokens = []
for sentence in df.lit_text:
    tokens = word_tokenize(sentence)
    if len(tokens) > max_vector_length:
        continue
    else:
        lit_tokens.append(tokens)

In [27]:
len(lit_tokens)

1378261

In [28]:
lit_vectors = list()
for i, tokens in enumerate(lit_tokens):
    padding_for = max_vector_length - len(tokens)
    vector = [np.zeros(target_dim)] * padding_for
    for word in tokens:
        try:
            vector.append(reduced_navec[word])
        except KeyError:
            try:
                vector.append(punkt_vectors[word])
            except KeyError:
                vector.append(reduced_navec["<unk>"])
    vector = np.array(vector, dtype=np.float16).reshape(max_vector_length * target_dim)
    lit_vectors.append(vector)
    print(len(lit_vectors))

Выходные данные были обрезаны до нескольких последних строк (5000).
1373262
1373263
1373264
1373265
1373266
1373267
1373268
1373269
1373270
1373271
1373272
1373273
1373274
1373275
1373276
1373277
1373278
1373279
1373280
1373281
1373282
1373283
1373284
1373285
1373286
1373287
1373288
1373289
1373290
1373291
1373292
1373293
1373294
1373295
1373296
1373297
1373298
1373299
1373300
1373301
1373302
1373303
1373304
1373305
1373306
1373307
1373308
1373309
1373310
1373311
1373312
1373313
1373314
1373315
1373316
1373317
1373318
1373319
1373320
1373321
1373322
1373323
1373324
1373325
1373326
1373327
1373328
1373329
1373330
1373331
1373332
1373333
1373334
1373335
1373336
1373337
1373338
1373339
1373340
1373341
1373342
1373343
1373344
1373345
1373346
1373347
1373348
1373349
1373350
1373351
1373352
1373353
1373354
1373355
1373356
1373357
1373358
1373359
1373360
1373361
1373362
1373363
1373364
1373365
1373366
1373367
1373368
1373369
1373370
1373371
1373372
1373373
1373374
1373375
1373376
1373377
1373

## Векторизация тг сообщений

In [14]:
tg_tokens = []
for sentence in df.tg_text:
    tokens = word_tokenize(sentence)
    if len(tokens) > max_vector_length:
        continue
    else:
        tg_tokens.append(tokens)

In [15]:
len(tg_tokens)

1382298

In [19]:
tg_vectors = list()
for i, tokens in enumerate(tg_tokens):
    padding_for = max_vector_length - len(tokens)
    vector = [np.zeros(target_dim)] * padding_for
    for word in tokens:
        try:
            vector.append(reduced_navec[word])
        except KeyError:
            try:
                vector.append(punkt_vectors[word])
            except KeyError:
                vector.append(reduced_navec["<unk>"])
    vector = np.array(vector, dtype=np.float16).reshape(max_vector_length * target_dim)
    tg_vectors.append(vector)
    print(len(tg_vectors))

Выходные данные были обрезаны до нескольких последних строк (5000).
1377299
1377300
1377301
1377302
1377303
1377304
1377305
1377306
1377307
1377308
1377309
1377310
1377311
1377312
1377313
1377314
1377315
1377316
1377317
1377318
1377319
1377320
1377321
1377322
1377323
1377324
1377325
1377326
1377327
1377328
1377329
1377330
1377331
1377332
1377333
1377334
1377335
1377336
1377337
1377338
1377339
1377340
1377341
1377342
1377343
1377344
1377345
1377346
1377347
1377348
1377349
1377350
1377351
1377352
1377353
1377354
1377355
1377356
1377357
1377358
1377359
1377360
1377361
1377362
1377363
1377364
1377365
1377366
1377367
1377368
1377369
1377370
1377371
1377372
1377373
1377374
1377375
1377376
1377377
1377378
1377379
1377380
1377381
1377382
1377383
1377384
1377385
1377386
1377387
1377388
1377389
1377390
1377391
1377392
1377393
1377394
1377395
1377396
1377397
1377398
1377399
1377400
1377401
1377402
1377403
1377404
1377405
1377406
1377407
1377408
1377409
1377410
1377411
1377412
1377413
1377414
1377

## Сохранение векторов

In [42]:
length = min(len(lit_vectors), len(tg_vectors))
length

1378261

In [44]:
lit_vectors_np = np.array(lit_vectors[:length])
tg_vectors_np = np.array(tg_vectors[:length])

In [47]:
np.save("/content/drive/MyDrive/CycleGAN_for_TST_problem/vectorized_lit.npy", lit_vectors_np)
np.save("/content/drive/MyDrive/CycleGAN_for_TST_problem/vectorized_tg.npy", tg_vectors_np)